In [ ]:
!pip install python-igraph
import pandas as pd
import igraph as ig

# Load node and edge data from CSV files
node_df = pd.read_csv('/content/Monuments_node.csv')
edge_df = pd.read_csv('/content/filtered_edges.csv')

# Create an empty graph
graph = ig.Graph(directed=False)

# Extract node IDs and labels
node_ids = node_df['id'].tolist()
node_labels = node_df['label'].tolist()

# Add nodes to the graph
graph.add_vertices(node_ids)
graph.vs['label'] = node_labels

# Extract and adjust edge pairs as tuples from the edge DataFrame
edge_pairs = list(zip(edge_df['source'], edge_df['target']))
adjusted_edge_pairs = [(source - 1, target - 1) for source, target in edge_pairs]

# Add edges to the graph with adjusted node IDs
graph.add_edges(adjusted_edge_pairs)

In [ ]:
# Add edge weights as a graph attribute (assuming 'Weight' column in edge_df)
graph.es['weight'] = edge_df['weight'].tolist()

degree_centrality = graph.strength(weights='weight')

# Calculate betweenness centrality (considering edge weights, weighted)
betweenness_centrality = graph.betweenness(weights='weight')

# Calculate closeness centrality (considering edge weights, weighted)
closeness_centrality = graph.closeness(weights='weight')


# Calculate closeness centrality (considering edge weights, weighted)
closeness_centrality = graph.closeness(weights='weight')

eigen_centrality = graph.evcent(weights='weight')

# Create a dictionary to store the centrality measures for each node
centrality_measures = {
    'Degree Centrality': degree_centrality,
    'Betweenness Centrality': betweenness_centrality,
    'Closeness Centrality': closeness_centrality,
    'Eigenvalue Centrality': eigen_centrality,
}

# Function to get top 5 nodes based on centrality values
def get_top_nodes(centrality_values):
    sorted_nodes = sorted(range(len(centrality_values)), key=lambda k: centrality_values[k], reverse=True)
    return sorted_nodes[:5]

# Print top 5 nodes for each centrality measure
for name, values in centrality_measures.items():
    print(f"Top 5 nodes for {name}:")
    top_nodes = get_top_nodes(values)
    for node_index in top_nodes:
        node_label = node_labels[node_index]
        centrality_value = values[node_index]
        print(f"Node Label: {node_label}, Centrality Value: {centrality_value}")
    print()

Top 5 nodes for Degree Centrality:
Node Label: National Rail Museum, Centrality Value: 376.0
Node Label: Delhi Haat -INA, Centrality Value: 330.0
Node Label: Sultan Ghari's Tomb, Centrality Value: 323.0
Node Label: Safdarjung Tomb, Centrality Value: 295.0
Node Label: sri Sri Radha Parthasarathi - ISKCON temple Sant Nagar, Centrality Value: 285.0

Top 5 nodes for Betweenness Centrality:
Node Label: Delhi Haat -INA, Centrality Value: 77.72023809523805
Node Label: National Rail Museum, Centrality Value: 36.17857142857142
Node Label: Majnu ka Tila, Centrality Value: 23.58333333333333
Node Label: Bharat Darshan Park, Centrality Value: 23.41666666666667
Node Label: Nehru Planetarium, Centrality Value: 16.38690476190476

Top 5 nodes for Closeness Centrality:
Node Label: Gurudwara Rakab Ganj, Centrality Value: 0.13333333333333333
Node Label: Amrit Udyan, Centrality Value: 0.13333333333333333
Node Label: National Museum, Centrality Value: 0.13333333333333333
Node Label: Gurudwara Bangla Sahib, 

In [ ]:
# Calculate the minimum and maximum degree centrality values
min_degree_centrality = min(degree_centrality)
max_degree_centrality = max(degree_centrality)

# Normalize the degree centrality values using Min-Max normalization
normalized_degree_centrality = [(value - min_degree_centrality) / (max_degree_centrality - min_degree_centrality) for value in degree_centrality]
max_betweenness_centrality = max(betweenness_centrality)

# Normalize the betweenness centrality values by dividing each value by the maximum
normalized_betweenness_centrality = [value / max_betweenness_centrality for value in betweenness_centrality]

# Create a dictionary to store the centrality measures for each node (including the normalized degree centrality and betweenness centrality)

centrality_data = pd.DataFrame({
    'Node ID': node_ids,
    'Node Label': node_labels,
   'Degree Centrality': normalized_degree_centrality,
    'Betweenness Centrality': normalized_betweenness_centrality,
    'Closeness Centrality': closeness_centrality,
    'Eigenvalue Centrality': eigen_centrality,
})

output_filename = 'metro_centralities_output.csv'
centrality_data.to_csv(output_filename, index=False)

print("Centrality data saved to:", output_filename)

Centrality data saved to: metro_centralities_output.csv


In [ ]:
ranks_df = pd.read_csv('/content/metro_centralities_output_Rank_Computed.csv')

top_nodes_dict = {}

# List to store all nodes that appear in the top 5 ranks for all centralities
top_nodes_all_centralities = []

# Centrality measures column names (modify these based on your CSV file)
centrality_columns = ['Degree_Rank', 'Betweeness_Rank', 'Closeness_Rank', 'EigenVector_Rank']

# Find top 5 nodes for each centrality measure
for centrality_measure in centrality_columns:
    top_nodes = ranks_df.nsmallest(10, centrality_measure)['Node ID'].tolist()
    top_nodes_dict[centrality_measure] = top_nodes
    top_nodes_all_centralities.extend(top_nodes)

# Find the nodes that appear in the top 5 ranks for all centralities
top_nodes_all_centralities = list(set(top_nodes_all_centralities))

# Print top 5 nodes for each centrality measure
for centrality_measure, top_nodes in top_nodes_dict.items():
    print(f"Top 10 nodes for {centrality_measure}:")
    for node_id in top_nodes:
        node_label = ranks_df.loc[ranks_df['Node ID'] == node_id, 'Node Label'].values[0]
        centrality_rank = ranks_df.loc[ranks_df['Node ID'] == node_id, centrality_measure].values[0]
        print(f"Node ID: {node_id}, Node Label: {node_label}, Rank: {centrality_rank}")
    print()

# Print nodes that appear in the top 5 ranks for all centralities
print("Nodes with top ranks in all centralities:")
for node_id in top_nodes_all_centralities:
    node_label = ranks_df.loc[ranks_df['Node ID'] == node_id, 'Node Label'].values[0]
    print(f"Node ID: {node_id}, Node Label: {node_label}")

Top 10 nodes for Degree_Rank:
Node ID: 29, Node Label: National Rail Museum, Rank: 1
Node ID: 7, Node Label: Delhi Haat -INA, Rank: 2
Node ID: 41, Node Label: Sultan Ghari's Tomb, Rank: 3
Node ID: 39, Node Label: Safdarjung Tomb, Rank: 4
Node ID: 16, Node Label: sri Sri Radha Parthasarathi - ISKCON temple Sant Nagar, Rank: 5
Node ID: 32, Node Label: Majnu ka Tila, Rank: 6
Node ID: 31, Node Label: Nehru Planetarium, Rank: 7
Node ID: 38, Node Label: Red Fort, Rank: 8
Node ID: 2, Node Label: Akshardham Temple, Rank: 9
Node ID: 12, Node Label: Hazrat Nizamuddin Shrine, Rank: 10

Top 10 nodes for Betweeness_Rank:
Node ID: 7, Node Label: Delhi Haat -INA, Rank: 1
Node ID: 29, Node Label: National Rail Museum, Rank: 2
Node ID: 32, Node Label: Majnu ka Tila, Rank: 3
Node ID: 46, Node Label: Bharat Darshan Park, Rank: 4
Node ID: 39, Node Label: Safdarjung Tomb, Rank: 5
Node ID: 31, Node Label: Nehru Planetarium, Rank: 6
Node ID: 9, Node Label: Gurudwara Rakab Ganj, Rank: 7
Node ID: 25, Node Labe

In [ ]:
# Add edge weights as a graph attribute (assuming 'Weight' column in edge_df)
graph.es['weight'] = edge_df['weight'].tolist()

degree_centrality = graph.strength()

# Calculate betweenness centrality (considering edge weights, weighted)
betweenness_centrality = graph.betweenness()

# Calculate closeness centrality (considering edge weights, weighted)
closeness_centrality = graph.closeness()


# Calculate closeness centrality (considering edge weights, weighted)
closeness_centrality = graph.closeness()

eigen_centrality = graph.evcent()

# Create a dictionary to store the centrality measures for each node
centrality_measures = {
    'Degree Centrality': degree_centrality,
    'Betweenness Centrality': betweenness_centrality,
    'Closeness Centrality': closeness_centrality,
    'Eigenvalue Centrality': eigen_centrality,
}

# Function to get top 5 nodes based on centrality values
def get_top_nodes(centrality_values):
    sorted_nodes = sorted(range(len(centrality_values)), key=lambda k: centrality_values[k], reverse=True)
    return sorted_nodes[:5]

# Print top 5 nodes for each centrality measure
for name, values in centrality_measures.items():
    print(f"Top 5 nodes for {name}:")
    top_nodes = get_top_nodes(values)
    for node_index in top_nodes:
        node_label = node_labels[node_index]
        centrality_value = values[node_index]
        print(f"Node Label: {node_label}, Centrality Value: {centrality_value}")
    print()

Top 5 nodes for Degree Centrality:
Node Label: National Rail Museum, Centrality Value: 42.0
Node Label: Delhi Haat -INA, Centrality Value: 41.0
Node Label: Gurudwara Rakab Ganj, Centrality Value: 39.0
Node Label: Amrit Udyan, Centrality Value: 39.0
Node Label: National Museum, Centrality Value: 39.0

Top 5 nodes for Betweenness Centrality:
Node Label: Delhi Haat -INA, Centrality Value: 50.58663177310109
Node Label: National Rail Museum, Centrality Value: 37.033699175816764
Node Label: Majnu ka Tila, Centrality Value: 31.977351916376307
Node Label: Nehru Planetarium, Centrality Value: 14.679892826695898
Node Label: Safdarjung Tomb, Centrality Value: 14.679892826695898

Top 5 nodes for Closeness Centrality:
Node Label: National Rail Museum, Centrality Value: 0.9565217391304348
Node Label: Delhi Haat -INA, Centrality Value: 0.9361702127659575
Node Label: Gurudwara Rakab Ganj, Centrality Value: 0.8979591836734694
Node Label: Amrit Udyan, Centrality Value: 0.8979591836734694
Node Label: Nat

In [ ]:
# Calculate the minimum and maximum degree centrality values
min_degree_centrality = min(degree_centrality)
max_degree_centrality = max(degree_centrality)

# Normalize the degree centrality values using Min-Max normalization
normalized_degree_centrality = [(value - min_degree_centrality) / (max_degree_centrality - min_degree_centrality) for value in degree_centrality]
max_betweenness_centrality = max(betweenness_centrality)

# Normalize the betweenness centrality values by dividing each value by the maximum
normalized_betweenness_centrality = [value / max_betweenness_centrality for value in betweenness_centrality]

# Create a dictionary to store the centrality measures for each node (including the normalized degree centrality and betweenness centrality)

centrality_data = pd.DataFrame({
    'Node ID': node_ids,
    'Node Label': node_labels,
   'Degree Centrality': normalized_degree_centrality,
    'Betweenness Centrality': normalized_betweenness_centrality,
    'Closeness Centrality': closeness_centrality,
    'Eigenvalue Centrality': eigen_centrality,
})

output_filename = 'metro_unweighted_centralities_output.csv'
centrality_data.to_csv(output_filename, index=False)

print("Centrality data saved to:", output_filename)

Centrality data saved to: metro_unweighted_centralities_output.csv
